## Core of the pipeline (PySpark)

In order to dockerise this component, the present Jupyter notebook should be converted into a Python script.

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkFiles
import numpy as np
from best_model import NonLinearModel, MovieLensDataset
import pickle

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/rafael/.netrc
wandb: Currently logged in as: rfg (rafaelgoncalvesua). Use `wandb login --relogin` to force relogin


In [2]:
KAFKA_TOPIC = "movielens"
KAFKA_BOOTSTRAP_SERVER = "localhost:9092"
NUM_USERS = 162541
NUM_MOVIES = 59047
EMBEDDING_DIM = 512
DROPOUT = 0.2
BATCH_SIZE = 64

In [3]:
spark = (
    SparkSession.builder.config(
        "spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0"
    )
    .appName("recommender")
    .getOrCreate()
)

sc = spark.sparkContext


24/06/19 03:33:19 WARN Utils: Your hostname, omen resolves to a loopback address: 127.0.1.1; using 192.168.1.122 instead (on interface wlo1)
24/06/19 03:33:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/rafael/Documentos/CAA/Project2/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/rafael/.ivy2/cache
The jars for the packages stored in: /home/rafael/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6fa46928-36b1-41b0-8753-b8c5c1ce7fe4;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 544ms :: artifacts dl 22ms
	::

In [4]:
raw_data = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
    .option("subscribe", KAFKA_TOPIC)
    .option("startingOffsets", "latest")
    .load()
)

data = (
    raw_data.selectExpr("CAST(value AS STRING) as value")
    .select(
        from_json(
            "value", "userId INT, movieId INT, rating DOUBLE, timestamp INT"
        ).alias("data")
    )
    .select("data.*")  # unpack dict
    .selectExpr("userId", "movieId", "rating")
)

sc.setLogLevel("OFF")

In [5]:
model = NonLinearModel(NUM_USERS, NUM_MOVIES, 512, 0.2)

# broadcast data buffer
broad_data_buffer = sc.broadcast([])

In [6]:
# load user2idx / item2idx mappings and scaler
user2idx = pickle.load(open("user2idx.pkl", "rb"))
item2idx = pickle.load(open("item2idx.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))

/home/rafael/.local/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# test scaler
scaler.transform(pd.DataFrame({"rating": [3.5, 4.0, 5.0]}))

array([[0.66666667],
       [0.77777778],
       [1.        ]])

In [8]:
len(user2idx), len(item2idx)

(142046, 23238)

In [9]:
# static test dataset and user/item index mapping
test_df = pd.read_csv("../data/ratings_test.csv", nrows=100000)

for user in test_df['userId']:
    if user not in user2idx:
        user2idx[user] = len(user2idx)

for item in test_df['movieId']:
    if item not in item2idx:
        item2idx[item] = len(item2idx)    

test_df['userId'] = test_df['userId'].map(user2idx)
test_df['movieId'] = test_df['movieId'].map(item2idx)
test_df['rating'] = scaler.transform(test_df[['rating']].values.reshape(-1, 1))
test_loader = DataLoader(MovieLensDataset(test_df), batch_size=BATCH_SIZE, shuffle=False)


/home/rafael/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [10]:
!rm model.pth
!cp model_original.pth model.pth

In [11]:
model.load_state_dict(torch.load("model.pth"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
mse = model.evaluate(test_loader, device)
# model.log({"test_loss": mse}) # initial loss
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}, MSE: {mse}")

RMSE: 2.9623490796321086, MSE: 8.775512069597202


### Query
We process the data for 5 hours **OR** until the data is exhausted **OR** until there are no computational resources available.

In [12]:
# delete query 'ratings_' if it exists
for q in spark.streams.active:
    if q.name == "ratings_":
        q.stop()


def fine_tune(batch_df, batch_id):
    if batch_df.count() == 0:
        print(f"[BATCH {batch_id}] Empty batch")
        return
    
    batch_pandas = batch_df.toPandas()

    for user in batch_pandas["userId"]:
        if user not in user2idx:
            user2idx[user] = len(user2idx)

    for item in batch_pandas["movieId"]:
        if item not in item2idx:
            item2idx[item] = len(item2idx)
    
    # update accumulated DataFrame with new batch as dict
    global broad_data_buffer
    broad_data_buffer.unpersist()
    data_buffer = broad_data_buffer.value.copy()
    data_buffer.extend(batch_pandas.to_dict(orient="records"))

    while len(data_buffer) < 20000:
        broad_data_buffer = sc.broadcast(data_buffer)
        print(f"[BATCH {batch_id}] Current buffer size: {len(data_buffer)}")
        return

    model.load_state_dict(torch.load("model.pth"))
    print(f"[BATCH {batch_id}] Model loaded")

    data_buffer_df = pd.DataFrame(data_buffer, index=None)
    data_buffer_df["rating"] = scaler.transform(data_buffer_df["rating"].values.reshape(-1, 1))
    print(f"[BATCH {batch_id}] Data scaled")

    data_buffer_df["userId"] = data_buffer_df["userId"].map(user2idx)
    data_buffer_df["movieId"] = data_buffer_df["movieId"].map(item2idx)

    dataset = MovieLensDataset(data_buffer_df)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print(f"[BATCH {batch_id}] All preprocessing done")

    print(f"[BATCH {batch_id}] Fitting Model")
    model.fit(train_loader, val_loader, test_loader, 10)
    print(f"[BATCH {batch_id}] Model fitted")

    torch.save(model.state_dict(), "model.pth")
    print(f"[BATCH {batch_id}] Model saved")

    # clear data buffer
    broad_data_buffer = sc.broadcast([])
    

query = (
    data.writeStream.trigger(processingTime="10 seconds")
    .foreachBatch(fine_tune)
    .start()
)

if not query.awaitTermination(18000):
    query.stop()

[BATCH 0] Empty batch


[BATCH 1] Current buffer size: 4219
[BATCH 2] Current buffer size: 17517
[BATCH 3] Model loaded
[BATCH 3] Data scaled


/home/rafael/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[BATCH 3] All preprocessing done
[BATCH 3] Fitting Model
Training on cuda
Epoch 1/10, Train Loss: 0.4212, Eval Loss: 0.0833
Epoch 2/10, Train Loss: 0.1091, Eval Loss: 0.0610
Epoch 3/10, Train Loss: 0.0904, Eval Loss: 0.0618
Epoch 4/10, Train Loss: 0.0755, Eval Loss: 0.0698
Epoch 5/10, Train Loss: 0.0665, Eval Loss: 0.0669
Epoch 6/10, Train Loss: 0.0588, Eval Loss: 0.0676
Epoch 7/10, Train Loss: 0.0528, Eval Loss: 0.0607
Epoch 8/10, Train Loss: 0.0475, Eval Loss: 0.0555
Epoch 9/10, Train Loss: 0.0435, Eval Loss: 0.0585
Epoch 10/10, Train Loss: 0.0408, Eval Loss: 0.0595
Test Loss: 0.0584
[BATCH 3] Model fitted
[BATCH 3] Model saved
[BATCH 4] Current buffer size: 11864


[BATCH 5] Model loaded
[BATCH 5] Data scaled


...

[BATCH 138] Model loaded
[BATCH 138] Data scaled


/home/rafael/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[BATCH 138] All preprocessing done
[BATCH 138] Fitting Model
Training on cuda
Epoch 1/10, Train Loss: 0.0458, Eval Loss: 0.0457
Epoch 2/10, Train Loss: 0.0376, Eval Loss: 0.0451
Epoch 3/10, Train Loss: 0.0313, Eval Loss: 0.0456
Epoch 4/10, Train Loss: 0.0278, Eval Loss: 0.0475
Epoch 5/10, Train Loss: 0.0248, Eval Loss: 0.0472
Epoch 6/10, Train Loss: 0.0232, Eval Loss: 0.0481
Epoch 7/10, Train Loss: 0.0217, Eval Loss: 0.0473
Epoch 8/10, Train Loss: 0.0212, Eval Loss: 0.0478
Epoch 9/10, Train Loss: 0.0203, Eval Loss: 0.0480
Epoch 10/10, Train Loss: 0.0199, Eval Loss: 0.0490
Test Loss: 0.0499
[BATCH 138] Model fitted


ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/home/rafael/Documentos/CAA/Project2/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/home/rafael/Documentos/CAA/Project2/venv/lib/python3.10/site-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/home/rafael/Documentos/CAA/Project2/venv/lib/python3.10/site-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_28607/3251011408.py", line 62, in fine_tune
    broad_data_buffer = sc.broadcast([])
  File "/home/rafael/Documentos/CAA/Project2/venv/lib/python3.10/site-packages/pyspark/context.py", line 1765, in broadcast
    return Broadcast(self, value, self._pickled_broadcast_vars)
  File "/home/rafael/Documentos/CAA/Project2/venv/lib/python3.10/s

[BATCH 138] Model saved
